# 3. ASCAT application
## Estimating the melt areas and last day of melt in Antarctic Peninsula using ASCAT data

## Introduction

Surface melt on the Antarctic Ice Sheet is a critical component of sea level rise projections, with estimates varying widely due to uncertainties surrounding the future of Antarctic ice shelves. 
While enhanced basal melt and iceberg calving currently dominate ice mass loss, surface melt, though playing a marginal role, can indirectly contribute by destabilizing ice shelves. Recent climate models predict a doubling of Antarctic surface melt by 2050, driven by atmospheric warming, potentially leading to significant increases in sea level rise contribution. Monitoring surface melt is essential for understanding these dynamics and refining projections.

Currently, three methods are utilized to estimate surface melt: automatic weather stations (AWSs), physics-based (regional) climate models (RCMs), and remote sensing. AWS observations facilitate the computation of surface energy balance and the estimation of excess energy available for surface melt. Initially, studies relied on temperature data from AWSs dating back to 1947, with subsequent research utilizing more advanced AWSs equipped with radiation sensors. While AWS observations are often considered "ground truth," their limitation lies in being point-based and restricted to a limited number of locations, thus inadequate for continentwide melt studies. Additionally, AWS locations are biased toward homogeneous snow surfaces, overlooking more complex surface types like blue ice, slush, and other wet surfaces.

The second method employs RCMs to estimate surface melt on a continental scale. However, the accuracy of RCMs depends on the resolution and precision of the forcing data, which often fall short for the Antarctic Ice Sheet. Furthermore, RCMs struggle to capture detailed melt features over low albedo regions due to their limited resolution.

Remote sensing provides an alternative for deriving long-term melt dynamics, offering various spatiotemporal resolutions. Various sensors, including radiometers, scatterometers, synthetic aperture radar, and optical sensors, enable the detection of wet surface conditions such as wet snow, slush, melt ponds, and streams. These sensors provide data for assessing the spatial and temporal patterns of surface melt, offering insights into its duration and distribution across the continent. 
Satellites equipped with remote sensing technology offer insights into the Antarctic surface across a broad spectrum of electromagnetic wavelengths, ranging from visible light to microwaves. These sensors detect surface reflectance, backscatter intensity (σ0), and brightness temperature (Tb). Changes in these parameters indicate the presence of liquid water, serving as indicators for its presence rather than directly observing the physical process of surface melt. Surface melt is essentially an energy conversion process, although thermal-infrared-derived surface temperature can pinpoint its occurrence at the melting point. Despite this distinction, the term "surface melt" is commonly used in remote sensing discourse. However, sensors primarily detect the presence of liquid water.
Furthermore, observations of backscatter intensity, brightness temperature, and surface reflectance are influenced by sensor characteristics and surface properties. Consequently, significant disparities can arise in surface melt estimates derived from different sensors, highlighting the importance of considering these factors when interpreting remote sensing data.
By exploiting satellite technology, researchers can enhance our understanding of Antarctic surface melt dynamics and their implications for future sea level rise.

In [ ]:
!pip install rasterio pyproj pandas cartopy

In [ ]:
import numpy as np
import rasterio
from rasterio.plot import show
from pyproj import Proj, transform
import matplotlib.pyplot as plt

## ASCAT data
ASCAT operates in C-band [5.255 GHz] and antennas are vertically-polarized only. In the southern hemisphere, ASCAT passes in the morning and in the evening. The 4.45 km enhanced resolution product is used, which was developed by the NASA Scatterometer Climate Record Pathfinder Project (available at: \url{http://www.scp.byu.edu}, last accessed on 24 May 2023). The TIFF files you are going to use have been downloaded using the Google Earth Engine tool.

In [ ]:
# Path to the georeferenced TIFF file
file_path = "Data/ASCAT/ascat_stack.tif"

# Open the TIFF file using rasterio
with rasterio.open(file_path) as src:
    # Display basic information about the file
    print("Number of bands:", src.count)
    print("Matrix dimensions:", src.shape)
    print("Spatial reference system:", src.crs)
    print("Geographic extent:", src.bounds)
    
    # Display the TIFF file
    show(src)

In [ ]:
# Definition of parameters
left, bottom, right, top = -2856900.0, 418300.0, -1441800.0, 1637600.0
rows, cols = 274, 318
resolution = 4450  # Step

# Creation of meshgrid matrices for x and y coordinates
x = np.linspace(left, right, cols)
y = np.linspace(bottom, top, rows)
xx, yy = np.meshgrid(x, y)

# Conversion of x and y coordinates to longitude and latitude (EPSG:3031)
in_proj = Proj(init='epsg:3031')
out_proj = Proj(proj='latlong', datum='WGS84')
lon, lat = transform(in_proj, out_proj, xx, yy)

# Path to the georeferenced TIFF file
file_path = "Data/ASCAT/ascat_stack.tif"

# Open the TIFF file using rasterio
with rasterio.open(file_path) as src:
    # Peninsula mask
    flipped = src.read(1)
    mask = np.where(np.isnan(flipped), np.nan, 1)
    # Plot of the src using latitude and longitude coordinates
    plt.figure(figsize=(12, 5))
    plt.imshow(src.read(1), cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()))
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Georeferenced TIFF Image')
    plt.colorbar(label='Pixel Value')
    plt.grid(visible=True)
    plt.show()


## Importing dates

Dates (one for each image) are stored in a separate .csv file. They have to be imported and converted in the correct format.

In [ ]:
import pandas as pd
from datetime import datetime

# Load CSV file
csv_file = "Data/ASCAT/ascat_date.csv"
data = pd.read_csv(csv_file)

# Extract dates column
dates_column = data['dates']

# Split each string in the Series by commas to get individual date strings
date_strings = dates_column.apply(lambda x: x.split(','))

# Flatten the list of lists to get a single list of date strings
date_strings_flat = [date for sublist in date_strings for date in sublist]

# Convert each string date to a datetime object and store them in a list
dates = [datetime.strptime(date, '%Y-%m-%d').date() for date in date_strings_flat]


# Print the first few dates as a sample
print("Sample of dates extracted from CSV:")
for date in dates[:5]:
    print(date)

# Now 'dates' list contains all the dates from the CSV file as datetime objects


## ASCAT images stack

In [ ]:
# Open the TIFF file using rasterio
with rasterio.open(file_path) as src:
    # Read all bands into a single numpy array stack
    stack_ascat = src.read()

# Check the shape of the stack
print("Shape of stack_ascat:", stack_ascat.shape)


## Annual stack 
Antarctic yearly stack: we are analyzing Antarctica, so we are going to create a annual stack with a 6-month offset. 
Now we will see how to create sub-stacks using defined dates.


In [ ]:

def find_indices_between_dates(dates, start_date, end_date):
    """
    Find the indices of dates in the dates array between start_date and end_date.

    Args:
        dates (numpy.ndarray): Array of dates.
        start_date (numpy.datetime64): Start date.
        end_date (numpy.datetime64): End date.

    Returns:
        numpy.ndarray: Array of indices of dates between start_date and end_date.
    """
    indices = np.where((dates >= start_date) & (dates <= end_date))[0]
    return indices

# 2018-19
start_date = np.datetime64("2018-06-01")
end_date = np.datetime64("2019-06-30")
indices_1819 = find_indices_between_dates(dates, start_date, end_date)
stack_1819 = stack_ascat[indices_1819]

# 2019-20
start_date = np.datetime64("2019-06-01")
end_date = np.datetime64("2020-06-30")
indices_1920 = find_indices_between_dates(dates, start_date, end_date)
stack_1920 = stack_ascat[indices_1920]

# 2020-21
start_date = np.datetime64("2020-06-01")
end_date = np.datetime64("2021-06-30")
indices_2021 = find_indices_between_dates(dates, start_date, end_date)
stack_2021 = stack_ascat[indices_2021]



## Stack reprojection

EPSG (European Petroleum Survey Group) codes are a standard way to reference coordinate systems and projections. In Antarctica, EPSG reprojections are commonly used to transform spatial data from one coordinate reference system to another. This is crucial for various applications such as mapping, geospatial analysis, and satellite imagery interpretation.

The most common EPSG code used in Antarctica is EPSG:3031, which represents the Antarctic Polar Stereographic projection. This projection is specifically designed for the polar regions and is optimized for accurate representation of areas near the poles. Here's how it works:

1. **Projection**: The Antarctic Polar Stereographic projection projects the spherical surface of the Earth onto a two-dimensional plane. It uses a stereographic projection method, which means that lines of longitude are projected as straight lines radiating from a point at the center of the projection, while lines of latitude are projected as concentric circles centered on that point.

2. **Datum**: The EPSG:3031 projection typically uses the WGS 84 datum, which is a global geodetic datum widely used in GPS systems and mapping applications. The WGS 84 datum defines the shape and size of the Earth's ellipsoid and serves as a reference for geospatial data.

3. **Parameters**: The EPSG:3031 projection has specific parameters that define how the projection is applied, such as the central meridian and the latitude of true scale. In the case of EPSG:3031, the central meridian is set to 0 degrees, and the latitude of true scale is set to -71 degrees (for the southern hemisphere).

4. **Transformation**: When data in a different coordinate reference system (CRS) needs to be displayed or analyzed alongside data in EPSG:3031, a reprojection is performed. This involves transforming the coordinates of each point in the original CRS to their equivalent coordinates in EPSG:3031. This transformation takes into account the mathematical formulas and parameters of the specific projections involved.

EPSG reprojections in Antarctica, particularly using EPSG:3031, enable accurate representation and analysis of spatial data in a way that is optimized for the unique characteristics of the polar region.

In [ ]:
# Plot yearly stacks
# Let's proceed with calculating the average of these stacks and plotting them in the selected projection.

# Assuming stack_1819, stack_1920, and stack_2021 are numpy arrays containing the stacked images
# Let's calculate the mean of each stack
mean_stack_1819 = np.nanmean(stack_1819, axis=0)
mean_stack_1920 = np.nanmean(stack_1920, axis=0)
mean_stack_2021 = np.nanmean(stack_2021, axis=0)

# Plot the georeferenced mean stacks
plt.figure(figsize=(12, 2))

# Plot mean_stack_1819
plt.subplot(1, 3, 1)
plt.imshow(mean_stack_1819,  cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=1)
plt.title("Mean Stack 2018-2019")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(visible=True)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=8)  # Adjust colorbar font size

# Plot mean_stack_1920
plt.subplot(1, 3, 2)
plt.imshow(mean_stack_1920,  cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=1)
plt.title("Mean Stack 2019-2020")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(visible=True)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=8)  # Adjust colorbar font size

# Plot mean_stack_2021
plt.subplot(1, 3, 3)
plt.imshow(mean_stack_2021,  cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=1)
plt.title("Mean Stack 2020-2021")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(visible=True)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=8)  # Adjust colorbar font size

plt.tight_layout()
plt.show()


## New stack creation based on winter season

Let's generate new stacks covering the period from June 1st to August 31st, known as the winter season.  These stacks are essential because the detection of melting (as we'll explore later) relies on variations starting from winter averages, characterized by very high backscatter.  We aim to identify differences in backscatter between the winter and non-winter periods: while ice typically exhibits high backscatter, areas experiencing water melt show lower backscatter readings.

In [ ]:
# 2018
start_date = np.datetime64("2018-06-01")
end_date = np.datetime64("2018-08-31")
indices_2018 = find_indices_between_dates(dates, start_date, end_date)
winter_2018 = stack_ascat[indices_2018]

# 2019
start_date = np.datetime64("2019-06-01")
end_date = np.datetime64("2019-08-31")
indices_2019 = find_indices_between_dates(dates, start_date, end_date)
winter_2019 = stack_ascat[indices_2019]

# 2020
start_date = np.datetime64("2020-06-01")
end_date = np.datetime64("2020-08-31")
indices_2020 = find_indices_between_dates(dates, start_date, end_date)
winter_2020 = stack_ascat[indices_2020]

# Winter's mean
mean_winter_2018 = np.nanmean(winter_2018, axis=0)
mean_winter_2019 = np.nanmean(winter_2019, axis=0)
mean_winter_2020 = np.nanmean(winter_2020, axis=0)

# Plot the georeferenced winter's mean
plt.figure(figsize=(12, 2))

# Plot mean_stack_1819
plt.subplot(1, 3, 1)
plt.imshow(mean_winter_2018,  cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=1)
plt.title("Winter 2018")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(visible=True)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=8)  # Adjust colorbar font size

# Plot mean_stack_1920
plt.subplot(1, 3, 2)
plt.imshow(mean_winter_2019,  cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=1)
plt.title("Winter 2019")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(visible=True)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=8)  # Adjust colorbar font size

# Plot mean_stack_2021
plt.subplot(1, 3, 3)
plt.imshow(mean_winter_2020,  cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=1)
plt.title("Winter 2020")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(visible=True)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=8)  # Adjust colorbar font size

plt.tight_layout()
plt.show()

# <font color='red'> Comparision for the year 2019: winter mean vs yearly mean </font> 
This is a comparison for the year 2019 on a large temporal scale, aimed at visualizing the macro areas where melt zones are concentrated.

In [ ]:
# Plot the georeferenced winter's mean
plt.figure(figsize=(12, 2))

# Plot mean_stack_1819
plt.subplot(1, 3, 1)
plt.imshow(mean_stack_1920,  cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=1)
plt.title("Mean Stack 2019-2020")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(visible=True)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=8)  # Adjust colorbar font size

# Plot winter 2019
plt.subplot(1, 3, 2)
plt.imshow(mean_winter_2019,  cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=1)
plt.title("Winter 2019")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(visible=True)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=8)  # Adjust colorbar font size

# Plot winter minus mean_stack_1920
plt.subplot(1, 3, 3)
plt.imshow(mean_winter_2019 - mean_stack_1920,  cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=0.5012)
plt.title("Difference winter - yearly 2019")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(visible=True)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=8)  # Adjust colorbar font size

## Computation of last doy melt 

To ensure the captured data corresponded to the end of the melt season, the study focuses exclusively on afternoon (6 PM) observations, as morning observations (6 AM) exhibit less melting [Trusel, L. D., et al. (2012). Antarctic surface melting dynamics: Enhanced perspectives from radar scatterometer data. Journal of Geophysical Research: Earth Surface, 117(F2).]. 
The melt detection algorithm proposed by Ashcraft [Ashcraft, I. S., & Long, D. G. (2006). Comparison of methods for melt detection over Greenland using active and passive microwave measurements. International Journal of Remote Sensing, 27(12), 2469-2488.] is implemented, assuming the presence of melt when the backscatter intensity falls below the annual winter mean by a threshold of 3 dB. Melt periods need to encompass four consecutive observations, with at least three out of the four observations indicating melt. Individual melt days are excluded, as a longer duration of melting is deemed necessary for aquifer formation. The last day of melt within the final selected melt period represents the "end of the melt season".

We are now going to calculate the last day of melt up to June 30th of each year. Specifically, we are interested in identifying the final day of melt during the first half of the year. We won't include late melt days in this analysis. "Late melt" in Antarctica typically refers to the period of time during the austral summer (December to February) when surface melting of ice and snow occurs later than usual. This phenomenon can have various causes, including weather patterns, atmospheric conditions, and changes in regional climate.

In [ ]:
# 2019
start_date = np.datetime64("2019-01-01")
end_date = np.datetime64("2019-06-30")
indices_19 = find_indices_between_dates(dates, start_date, end_date)
stack19 = stack_ascat[indices_19]
dates19 = dates[indices_19[0]:indices_19[len(indices_19)-1]]

# 2020
start_date = np.datetime64("2020-01-01")
end_date = np.datetime64("2020-06-30")
indices_20 = find_indices_between_dates(dates, start_date, end_date)
stack20 = stack_ascat[indices_20]
dates20 = dates[indices_20[0]:indices_20[len(indices_20)-1]]

# 2021
start_date = np.datetime64("2021-01-01")
end_date = np.datetime64("2021-06-30")
indices_21 = find_indices_between_dates(dates, start_date, end_date)
stack21 = stack_ascat[indices_21]
dates21 = dates[indices_21[0]:indices_21[len(indices_21)-1]]

# <font color='red'> Application of melt threshold </font> 

This section involves the implementation of the actual melt algorithm: for each pixel, if the difference of 3 dB between the winter period and the i-th day is exceeded, the pixel is considered to have experienced melt on that day. The comparison is made against the winter average of that pixel.
For each image, each pixel is considered melt if:$\sigma_0 \leq \bar{\sigma}_{\text{0 winter}} - 3 \text{db} $, where $\bar{\sigma}_{\text{0 winter}}$ is the winter mean.

In [ ]:
# For the year 2019, due to sensor faults, we have significantly fewer images compared to the other two years.
# Threshold melt of 3 db between winter average and current image is expressed as float number
threshold = 10 ** (-3 * 0.1)

# Define the dimensions of the matrices
rows, cols = stack19[0].shape  # Assuming images in the stack have the same dimensions

# Create containers for matrices
melt19 = [np.zeros((rows, cols)) for _ in range(len(stack19))]
melt20 = [np.zeros((rows, cols)) for _ in range(len(stack20))]
melt21 = [np.zeros((rows, cols)) for _ in range(len(stack21))]

# Calculate the differences and assign appropriate values
for i in range(len(stack19)):
    melt19[i][np.where(mean_winter_2018 - stack19[i] > threshold)] = 1

for i in range(len(stack20)):
    melt20[i][np.where(mean_winter_2019 - stack20[i] > threshold)] = 1

for i in range(len(stack21)):
    melt21[i][np.where(mean_winter_2020 - stack21[i] > threshold)] = 1

# Sum the images in melt19, melt20, and melt21
# Calculate the total number of days with melt.
sum_melt19 = np.sum(melt19, axis=0)
sum_melt20 = np.sum(melt20, axis=0)
sum_melt21 = np.sum(melt21, axis=0)

In [ ]:
# Number of times melt up to 30th June (note that we do not have an ascat image per day)
# The colorbar has been saturated up to the number of images avaible up to 30th june: it is a relative number whose maximum value coincides with the number of images we have for that particular year. 


# Mask application
sum_melt19_masked = np.ma.masked_where(np.isnan(mask), sum_melt19)
sum_melt20_masked = np.ma.masked_where(np.isnan(mask), sum_melt20)
sum_melt21_masked = np.ma.masked_where(np.isnan(mask), sum_melt21)

# Matrices plot
fig, axs = plt.subplots(1, 3, figsize=(15, 2))

# Plot per sum_melt19
img1 = axs[0].imshow(sum_melt19_masked, cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=len(dates19))
axs[0].set_title('Number of times melt: 2019')
axs[0].set_xlabel('Longitude')
axs[0].set_ylabel('Latitude')
cbar1 = fig.colorbar(img1, ax=axs[0])
cbar1.set_label('N melt days')

# Plot per sum_melt20
img2 = axs[1].imshow(sum_melt20_masked, cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=len(dates20))
axs[1].set_title('Number of times melt: 2020')
axs[1].set_xlabel('Longitude')
axs[1].set_ylabel('Latitude')
cbar2 = fig.colorbar(img2, ax=axs[1])
cbar2.set_label('N melt days')

# Plot per sum_melt21
img3 = axs[2].imshow(sum_melt21_masked, cmap='jet', extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=len(dates21))
axs[2].set_title('Number of times melt: 2021')
axs[2].set_xlabel('Longitude')
axs[2].set_ylabel('Latitude')
cbar3 = fig.colorbar(img3, ax=axs[2])
cbar3.set_label('N melt days')

plt.tight_layout()
plt.show()



In [ ]:
# Conversion from date to doy (day of year)

import datetime

doy19 = [date.timetuple().tm_yday for date in dates19]
doy20 = [date.timetuple().tm_yday for date in dates20]
doy21 = [date.timetuple().tm_yday for date in dates21]

## Last DoY melt

In [ ]:
# For each pixel, it searches for the last positive melt value, and from that index, I derive the last day of the year (DOY).

# Calculating the indices of the last occurrence of 1 for each element i, j in the matrices
indices_19 = np.zeros_like(melt19[0])
indices_20 = np.zeros_like(melt20[0])
indices_21 = np.zeros_like(melt21[0])

for i in range(melt19[0].shape[0]):
    for j in range(melt19[0].shape[1]):
        for k in range(len(melt19)):
            if melt19[k][i, j] == 1:
                indices_19[i, j] = k + 1

for i in range(melt20[0].shape[0]):
    for j in range(melt20[0].shape[1]):
        for k in range(len(melt20)):
            if melt20[k][i, j] == 1:
                indices_20[i, j] = k + 1

for i in range(melt21[0].shape[0]):
    for j in range(melt21[0].shape[1]):
        for k in range(len(melt21)):
            if melt21[k][i, j] == 1:
                indices_21[i, j] = k + 1

last_doy19 = np.zeros_like(melt19[0])
last_doy20 = np.zeros_like(melt20[0])
last_doy21 = np.zeros_like(melt21[0])

# Last doy melt 2019
for i in range(indices_19.shape[0]):
    for j in range(indices_19.shape[1]):
        if indices_19[i, j] > 0:
            ind_int = int(indices_19[i, j] - 1)
            last_doy19[i, j] = doy19[ind_int]

# Last doy melt 2020
for i in range(indices_20.shape[0]):
    for j in range(indices_20.shape[1]):
        if indices_20[i, j] > 0:
            ind_int = int(indices_20[i, j] - 1)
            last_doy20[i, j] = doy20[ind_int]


# Last doy melt 2021
for i in range(indices_21.shape[0]):
    for j in range(indices_21.shape[1]):
        if indices_21[i, j] > 0:
            ind_int = int(indices_21[i, j] - 1)
            last_doy21[i, j] = doy21[ind_int]


In [ ]:
# Last doy melt: up to 30th June, plot

# Saturation value

sat = 80
style = 'gist_ncar'

# Mask application
doy_melt19_masked = np.ma.masked_where(np.isnan(mask), last_doy19)
doy_melt20_masked = np.ma.masked_where(np.isnan(mask), last_doy20)
doy_melt21_masked = np.ma.masked_where(np.isnan(mask), last_doy21)

# Plot
fig, axs = plt.subplots(1, 3, figsize=(15, 2))

# Last doy melt 2019
img1 = axs[0].imshow(doy_melt19_masked, cmap=style, extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=sat)
axs[0].set_title('Last doy melt: 2019')
axs[0].set_xlabel('Longitude')
axs[0].set_ylabel('Latitude')
cbar1 = fig.colorbar(img1, ax=axs[0])
cbar1.set_label('Last DOY melt')

# Last doy melt 2020
img2 = axs[1].imshow(doy_melt20_masked, cmap=style, extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=sat)
axs[1].set_title('Last doy melt: 2020')
axs[1].set_xlabel('Longitude')
axs[1].set_ylabel('Latitude')
cbar2 = fig.colorbar(img2, ax=axs[1])
cbar2.set_label('Last DOY melt')

# Last doy melt 2021
img3 = axs[2].imshow(doy_melt21_masked, cmap=style, extent=(lon.min(), lon.max(), lat.min(), lat.max()), vmin=0, vmax=sat)
axs[2].set_title('Last doy melt: 2021')
axs[2].set_xlabel('Longitude')
axs[2].set_ylabel('Latitude')
cbar3 = fig.colorbar(img3, ax=axs[2])
cbar3.set_label('Last DOY melt')

plt.tight_layout()
plt.show()



## Import model data 

A Regional Climate Model (RCM) is a type of atmospheric model designed to simulate climate conditions over a specific region at a certain spatial resolution. RCMs contain mathematical equations that represent physical processes such as radiation, convection, and advection, similar to global climate models but focused on a smaller geographical area.

RACMO (Regional Atmospheric Climate Model) is a specific RCM developed by the Royal Netherlands Meteorological Institute (KNMI) primarily to study polar regions, particularly Antarctica and Greenland. RACMO aims to provide detailed information about atmospheric variables such as temperature, precipitation, wind patterns, and humidity over these regions.
To create RACMO, scientists collect observational data from various sources such as weather stations, satellites, and field campaigns. This observational data is used to initialize the model and to validate its performance. The model equations are then solved numerically using computer simulations to predict how the climate will evolve over time under different scenarios and forcings.
RACMO incorporates processes specific to polar regions, such as ice melt, snow accumulation, and atmospheric circulation patterns, to provide accurate simulations of the climate in these areas. It can be used to study the impacts of climate change on polar ice sheets, sea ice extent, and regional climate variability.

in the context of RACMO, melt flux refers to the rate at which snow or ice melts and releases liquid water into the environment. It's measured as the volume of water produced per unit area over a specific time period, such as millimeters per day. Factors like temperature, solar radiation, wind, and topography influence melt flux. Understanding melt flux is crucial for assessing freshwater resources, hydrological cycles, and the impacts of climate change on snow and ice dynamics.

We are going to use meltflux data which are statistically downscaled to 2 km  x 2 km from the output of the regional climate model RACMO2.3p2 (27 km x 27 km resolution) in the Antarctic region.
[Noël, B. et al. (2023). Higher Antarctic ice sheet accumulation and surface melt rates revealed at 2 km resolution. Nature communications, 14(1), 7949.]

In [ ]:
# Now we are going to use the model data to plot the cumulative melt [mm w.e./y] over the year for each pixel.
# Model images parameters

# Path to the georeferenced TIFF file
file_path = "Data/ASCAT/Melt_model_2019.tif"

# Open the TIFF file using rasterio
with rasterio.open(file_path) as src:
    # Display basic information about the file
    print("Number of bands:", src.count)
    print("Matrix dimensions:", src.shape)
    print("Spatial reference system:", src.crs)
    print("Geographic extent:", src.bounds)

# <font color='red'> Visual comparision with melt flux  </font> 
You will see how in 2019, there is less effective functioning due to fewer images available, whereas in other years, there is a good correspondence between areas with high melt rates and areas where the last Day of Year (DoY) melt is higher.

In [ ]:
# Comparision with melt flux rate estimated by the model: 2019

# Note that model's images have a better resolution than ASCAT

# Definition of parameters

left, bottom, right, top = -2794000.0, 30000.0, -1304000.0, 2508000.0
rows, cols = 745, 1239
resolution = 2000  # Step

# Creation of meshgrid matrices for x and y coordinates
x_model = np.linspace(left, right, cols)
y_model = np.linspace(bottom, top, rows)
xx_model, yy_model = np.meshgrid(x_model, y_model)

# Melt flux saturation value
sat_model = 600


# Open the TIFF file using rasterio
file_path = "Data/ASCAT/Melt_model_2019.tif"

# Plotting the model melt flux for 2019
with rasterio.open(file_path) as src:
    plt.figure(figsize=(14, 7))  # Adjust figure size as needed
    plt.subplot(1, 2, 1)  # First subplot
    plt.imshow(src.read(1), cmap=style, extent=(xx_model.min(), xx_model.max(), yy_model.min(), yy_model.max()), vmin=0, vmax=sat_model)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Model Melt Flux: 2019')
    plt.colorbar(label='Melt Flux')
    plt.grid(visible=True)

# Plotting the last day of year (doy) melt for 2019
plt.subplot(1, 2, 2)  # Second subplot
img2 = plt.imshow(doy_melt19_masked, cmap=style, extent=(xx.min(), xx.max(), yy.min(), yy.max()), vmin=0, vmax=sat)
plt.title('Last Day of Year Melt: 2019')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
cbar2 = plt.colorbar(img2)
cbar2.set_label('Last DoY melt')
plt.grid(visible=True)

plt.tight_layout()
plt.show()

In [ ]:
# Comparision with melt flux rate estimated by the model: 2020

# Note that model's images have a better resolution than ASCAT

# Open the TIFF file using rasterio
file_path = "Data/ASCAT/Melt_model_2020.tif"

# Plotting the model melt flux for 2019
with rasterio.open(file_path) as src:
    plt.figure(figsize=(14, 7))  # Adjust figure size as needed
    plt.subplot(1, 2, 1)  # First subplot
    plt.imshow(src.read(1), cmap=style, extent=(xx_model.min(), xx_model.max(), yy_model.min(), yy_model.max()), vmin=0, vmax=sat_model)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Model Melt Flux: 2020')
    plt.colorbar(label='Melt Flux')
    plt.grid(visible=True)

# Plotting the last day of year (doy) melt for 2019
plt.subplot(1, 2, 2)  # Second subplot
img2 = plt.imshow(doy_melt20_masked, cmap=style, extent=(xx.min(), xx.max(), yy.min(), yy.max()), vmin=0, vmax=sat)
plt.title('Last Day of Year Melt: 2020')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
cbar2 = plt.colorbar(img2)
cbar2.set_label('Last DoY melt')
plt.grid(visible=True)

plt.tight_layout()
plt.show()

In [ ]:
# --- neat Antarctic melt plotting utilities (Cartopy) ---

import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import LinearSegmentedColormap
from typing import Optional, Tuple, Union

# Default map projections
MAP_PROJ = ccrs.Stereographic(central_longitude=0, central_latitude=-90)
PLATE_CARREE = ccrs.PlateCarree()
SOUTH_POLAR_STEREO = ccrs.SouthPolarStereo()

# Handy named colormaps
COLORMAPS = {
    "melt_classic": "YlOrRd",
    "melt_modern": "plasma",
    "ice_blue": "Blues",
    "thermal": "hot",
    "ocean": "ocean",
    "viridis": "viridis",
    "turbo": "turbo",
}


def create_custom_melt_colormap() -> LinearSegmentedColormap:
    """Dark blue -> light blue -> yellow -> orange -> red."""
    colors = ["#0d47a1", "#42a5f5", "#81d4fa", "#fff176", "#ffb74d", "#ff7043", "#d32f2f"]
    return LinearSegmentedColormap.from_list("melt_gradient", colors, N=256)


def _auto_vlims(a: np.ndarray, vmin: Optional[float], vmax: Optional[float]) -> Tuple[float, float]:
    """Percentile stretch if vmin/vmax not provided."""
    if vmin is None or vmax is None:
        finite = np.asarray(a, dtype=float)
        finite = finite[np.isfinite(finite)]
        if finite.size == 0:
            return 0.0, 1.0
        p2, p98 = np.percentile(finite, [2, 98])
        vmin = p2 if vmin is None else vmin
        vmax = p98 if vmax is None else vmax
        if vmin == vmax:
            vmax = vmin + 1.0
    return float(vmin), float(vmax)


def _resolve_cmap(cmap: Union[str, LinearSegmentedColormap]) -> Union[str, LinearSegmentedColormap]:
    if isinstance(cmap, str) and cmap in COLORMAPS:
        return COLORMAPS[cmap]
    return cmap


def plot_melt_cartopy(
    melt_data: np.ndarray,
    *,
    # Either lon/lat (in degrees) OR projected x/y (SouthPolarStereo meters)
    lon: Optional[np.ndarray] = None,
    lat: Optional[np.ndarray] = None,
    x: Optional[np.ndarray] = None,
    y: Optional[np.ndarray] = None,
    title: str = "Melt Days",
    year: Optional[Union[int, str]] = None,
    cmap: Union[str, LinearSegmentedColormap] = "plasma",
    vmin: Optional[float] = None,
    vmax: Optional[float] = None,
    mask_zeros: bool = True,
    coastline_style: str = "clean",  # "clean" | "minimal" | "detailed"
    colorbar_label: str = "Cumulative Melt Days",
    figsize: Tuple[int, int] = (12, 10),
    background_face: str = "#ffffff",
) -> Tuple[plt.Figure, plt.Axes, any]:
    """
    Plot Antarctic melt data in a South Polar projection with a clean aesthetic.

    Pass either (lon, lat) in degrees OR (x, y) already in SouthPolarStereo meters.
    - If lon/lat are given, the data are assumed on a geographic grid and drawn with PlateCarree().
    - If x/y are given, the data are assumed in SouthPolarStereo and drawn in that CRS.

    Returns (fig, ax, mappable).
    """
    A = np.asarray(melt_data)
    if A.ndim != 2:
        raise ValueError(f"melt_data must be 2D, got shape {A.shape}")

    # Choose data CRS and coordinate arrays
    if lon is not None and lat is not None:
        data_crs = PLATE_CARREE
        X, Y = np.asarray(lon), np.asarray(lat)
    elif x is not None and y is not None:
        data_crs = SOUTH_POLAR_STEREO
        X, Y = np.asarray(x), np.asarray(y)
    else:
        raise ValueError("Provide either (lon, lat) OR (x, y) coordinates.")

    # Mask zeros if desired (helps avoid background color influence)
    if mask_zeros:
        A = np.ma.masked_where(A == 0, A)

    vmin, vmax = _auto_vlims(A, vmin, vmax)
    cmap = _resolve_cmap(cmap)

    # Prepare coordinates for pcolormesh: accept 1D or 2D
    if X.ndim == 1 and Y.ndim == 1:
        # Expect len(X) == A.shape[1] and len(Y) == A.shape[0]
        if len(X) != A.shape[1] or len(Y) != A.shape[0]:
            raise ValueError(
                f"When X/Y are 1D, lengths must match data shape (Y: {len(Y)} vs {A.shape[0]}, X: {len(X)} vs {A.shape[1]})."
            )
        XX, YY = np.meshgrid(X, Y)
    elif X.ndim == 2 and Y.ndim == 2:
        XX, YY = X, Y
        if XX.shape != A.shape or YY.shape != A.shape:
            raise ValueError(f"2D X/Y must match data shape: X {XX.shape}, Y {YY.shape}, A {A.shape}.")
    else:
        raise ValueError("X and Y must be both 1D or both 2D, with consistent shapes.")

    # Figure & axis
    fig, ax = plt.subplots(1, 1, figsize=figsize, subplot_kw={"projection": MAP_PROJ})
    fig.patch.set_facecolor(background_face)
    ax.set_facecolor("#f8f9fa")  # soft gray to avoid color clashes

    # Draw data (pcolormesh is robust for geolocated grids)
    m = ax.pcolormesh(XX, YY, A, transform=data_crs, shading="auto", cmap=cmap, vmin=vmin, vmax=vmax, zorder=2)

    # Coastlines / features
    if coastline_style == "clean":
        ax.add_feature(cfeature.COASTLINE, linewidth=1.2, edgecolor="white", facecolor="none", zorder=3)
    elif coastline_style == "minimal":
        ax.add_feature(cfeature.COASTLINE, linewidth=0.8, edgecolor="lightgray", facecolor="none", zorder=3)
    elif coastline_style == "detailed":
        ax.add_feature(cfeature.COASTLINE, linewidth=1.6, edgecolor="white", facecolor="none", zorder=3)
        # Optional: try ice shelves if Natural Earth data available
        try:
            ice_shelves = cfeature.NaturalEarthFeature(
                "physical", "antarctic_ice_shelves_polys", "50m", facecolor="none", edgecolor="lightgray", linewidth=1.0
            )
            ax.add_feature(ice_shelves, zorder=3)
        except Exception:
            pass

    # Subtle graticule lines (labels on polar projections can be messy, so keep unlabeled)
    gl = ax.gridlines(crs=PLATE_CARREE, draw_labels=False, linewidth=0.5, linestyle=":", color="gray", alpha=0.4)
    ax.set_extent([-180, 180, -90, -55], crs=PLATE_CARREE)  # crop to Antarctica+Southern Ocean rim

    # Title
    t = f"{title}" if year is None else f"{title} {year}"
    ax.set_title(t, fontsize=16, pad=18, fontweight="bold", color="#2c3e50")

    # Colorbar
    cbar = plt.colorbar(m, ax=ax, orientation="horizontal", pad=0.06, shrink=0.78, aspect=40)
    cbar.ax.tick_params(labelsize=11)
    if colorbar_label:
        cbar.set_label(colorbar_label, fontsize=12, fontweight="bold", color="#2c3e50")

    plt.tight_layout()
    return fig, ax, m


def print_data_statistics(data: np.ndarray, name: str = "Dataset") -> None:
    """Quick stats for sanity checks."""
    a = np.asarray(data)
    print("\n" + "=" * 50)
    print(f"{name} Statistics")
    print("=" * 50)
    print(f"Shape: {a.shape}")
    print(f"Dtype: {a.dtype}")
    print(f"Total pixels: {a.size:,}")
    nz = int(np.count_nonzero(a))
    print(f"Non-zero pixels: {nz:,}")
    print(f"Zero pixels: {a.size - nz:,}")
    finite = a[np.isfinite(a)]
    if finite.size:
        print(f"Min: {np.nanmin(finite):.2f}")
        print(f"Max: {np.nanmax(finite):.2f}")
        print(f"Mean: {np.nanmean(finite):.2f}")
        print(f"Median: {np.nanmedian(finite):.2f}")
        print(f"Std: {np.nanstd(finite):.2f}")
    else:
        print("All values are non-finite.")
    print("=" * 50)


# Backwards-compatible alias if you prefer the old name
plot_melt_data = plot_melt_cartopy
